In [1]:
import pandas as pd # library for data analsysis
from bs4 import BeautifulSoup # library for pulling data out of HTML and XML files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import re
print('Libraries imported.')

Libraries imported.


Obtain the Neighbourhood Detials For Toronto 

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

In [3]:
table_can = soup.find('table')
col = table_can.find_all('td')

In [4]:
elem_cnt = len(col)

# create three lists to store table data
postcode = []
borough = []
neighborhood = []

Data Cleaning

In [5]:
for i in range(0, elem_cnt):
	postcode.append(col[i].find("b").text.strip())
	span=col[i].find("span")
	if "href" in str(span):
		bor=str(span).split("<br/>")
		bor2=re.sub(r'<.*?>','',bor[0])
		ne=re.sub(r'<.*?>','',bor[1])
		borough.append(bor2.strip())
		neighborhood.append(ne.strip())
	else:
		borough.append("Not assigned")
		neighborhood.append("Not assigned")



Formatting

In [6]:
toronto_df = pd.DataFrame({"PostalCode": postcode,
                           "Borough": borough,
                           "Neighborhood": neighborhood})



In [7]:
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,(Parkwoods)
3,M4A,North York,(Victoria Village)
4,M5A,Downtown Toronto,(Regent Park / Harbourfront)


neighbourhood Exploration

In [8]:
# Normalize data and transform per given requirements
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df.loc[toronto_df.Neighborhood == 'Not assigned', "Neighborhood"] = toronto_df.Borough



In [9]:
#Group data by Postcode & Borough
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))


# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]

Select the Specific Postcodes

In [10]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
neighborhoods_df = pd.DataFrame(columns=column_names)

neighborhoods_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

In [11]:
for postcode in neighborhoods_list:
    neighborhoods_df = neighborhoods_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
print(neighborhoods_df)

   PostalCode  ...                                       Neighborhood
0         M5G  ...                               (Central Bay Street)
1         M2H  ...                                (Hillcrest Village)
2         M4B  ...                 (Parkview Hill / Woodbine Gardens)
3         M1J  ...                              (Scarborough Village)
4         M4G  ...                                          (Leaside)
5         M1R  ...                               (Wexford / Maryvale)
6         M9V  ...  (South Steeles / Silverstone / Humbergate / Ja...
7         M9L  ...                                    (Humber Summit)
8         M5V  ...  (CN Tower / King and Spadina / Railway Lands /...
9         M1B  ...                                  (Malvern / Rouge)
10        M5A  ...                       (Regent Park / Harbourfront)

[11 rows x 3 columns]


Lat Long location

In [12]:
import io
import requests

url = "https://cocl.us/Geospatial_data"
lat_long = requests.get(url).text
lat_long_df=pd.read_csv(io.StringIO(lat_long))
lat_long_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
lat_long_df = lat_long_df.rename(columns={'Postal Code': 'PostalCode'})
lat_long_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Combine the two dataframes 

In [14]:
toronto_pc = pd.merge(toronto_df,lat_long_df, on='PostalCode')
toronto_pc = toronto_pc.rename(columns={'Neighbourhood':'Neighborhood'})
toronto_pc.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,(Parkwoods),43.753259,-79.329656
1,M4A,North York,(Victoria Village),43.725882,-79.315572
2,M5A,Downtown Toronto,(Regent Park / Harbourfront),43.654260,-79.360636
3,M6A,North York,(Lawrence Manor / Lawrence Heights),43.718518,-79.464763
4,M7A,Queen's Park,(Ontario Provincial Government),43.662301,-79.389494


In [15]:
import json 

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library


In [16]:
df4 = toronto_pc[toronto_pc['Borough'].str.contains('Toronto',regex=False)]
df4

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,(Regent Park / Harbourfront),43.654260,-79.360636
9,M5B,Downtown Toronto,"(Garden District, Ryerson)",43.657162,-79.378937
15,M5C,Downtown Toronto,(St. James Town),43.651494,-79.375418
19,M4E,East Toronto,(The Beaches),43.676357,-79.293031
20,M5E,Downtown Toronto,(Berczy Park),43.644771,-79.373306
24,M5G,Downtown Toronto,(Central Bay Street),43.657952,-79.387383
25,M6G,Downtown Toronto,(Christie),43.669542,-79.422564
30,M5H,Downtown Toronto,(Richmond / Adelaide / King),43.650571,-79.384568
31,M6H,West Toronto,(Dufferin / Dovercourt Village),43.669005,-79.442259
36,M5J,Downtown Toronto,(Harbourfront East / Union Station / Toronto I...,43.640816,-79.381752


In [17]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [18]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df4['Latitude'],df4['Longitude'],df4['Borough'],df4['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

k-means clustering

In [19]:
k=5
toronto_clustering = df4.drop(['PostalCode','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
df4.insert(0, 'Cluster Labels', kmeans.labels_)

In [20]:
df4

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,4,M5A,Downtown Toronto,(Regent Park / Harbourfront),43.654260,-79.360636
9,4,M5B,Downtown Toronto,"(Garden District, Ryerson)",43.657162,-79.378937
15,4,M5C,Downtown Toronto,(St. James Town),43.651494,-79.375418
19,2,M4E,East Toronto,(The Beaches),43.676357,-79.293031
20,4,M5E,Downtown Toronto,(Berczy Park),43.644771,-79.373306
24,4,M5G,Downtown Toronto,(Central Bay Street),43.657952,-79.387383
25,1,M6G,Downtown Toronto,(Christie),43.669542,-79.422564
30,4,M5H,Downtown Toronto,(Richmond / Adelaide / King),43.650571,-79.384568
31,3,M6H,West Toronto,(Dufferin / Dovercourt Village),43.669005,-79.442259
36,4,M5J,Downtown Toronto,(Harbourfront East / Union Station / Toronto I...,43.640816,-79.381752


In [22]:
import numpy as np


In [23]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df4['Latitude'], df4['Longitude'], df4['Neighborhood'], df4['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters